In [ ]:
import numpy as np
from sklearn.cross_validation import train_test_split
from IPython import display
from sklearn.metrics import roc_curve, auc, confusion_matrix
from __future__ import division
import matplotlib.pylab as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

filename = ""
vp_table = np.zeros((2,10))
fp_table = np.zeros((2,10))
fn_table = np.zeros((2,10))
vn_table = np.zeros((2,10))
for i in range(10):
    filename = 'DataBases/LBPTruncadoRaw'+str(i+1)+'.csv'
    print "Reading DataBase with partition: "+str(i+1)
    print filename
    csv = np.genfromtxt (filename, delimiter=',')
    
    #balanced classes. For not balanced classes, select all the rows of the csv
    Y = csv[350:,0]
    X = csv[350:,1:]
    
    trainSize = 0.6
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=trainSize)
    
    #Dimensionality reduction step. If not desire, just comment out this block
    clf = RandomForestClassifier()
    clf = clf.fit(X_train, Y_train)
    model = SelectFromModel(clf, prefit=True)
    X_train = model.transform(X_train)
    X_test = model.transform(X_test)
    
    print "Classifying with RandomForest"
    #train 5 random forest to avoid random noise
    vn_aux = np.zeros(5)
    vp_aux = np.zeros(5)
    fp_aux = np.zeros(5)
    fn_aux = np.zeros(5)
    for j in range(5):
        print "Training Random Forest "+str(j+1)
        classifier = RandomForestClassifier(n_estimators=100, max_depth=None, class_weight='balanced')
        classifier.fit(X_train, Y_train)
        print "Computing confusion matrix"
        predicted_test_labels = classifier.predict_proba(X_test)[:,1]
        [[VN,FP],[FN,VP]]=confusion_matrix(Y_test.astype(bool),(predicted_test_labels>0.5).astype(bool)).astype(float)
        #save current confusion matrix of later use
        vn_aux[j] = VN
        vp_aux[j] = VP
        fp_aux[j] = FP
        fn_aux[j] = FN
    print "Saving classification data for the data base "+str(i+1)
    print ""
    #save mean and std of confusion matrix
    vn_table[0,i] = np.mean(vn_aux)#mean
    vp_table[0,i] = np.mean(vp_aux)
    fp_table[0,i] = np.mean(fp_aux)
    fn_table[0,i] = np.mean(fn_aux)
    
    vn_table[1,i] = np.std(vn_aux)#desvest
    vp_table[1,i] = np.std(vp_aux)
    fp_table[1,i] = np.std(fp_aux)
    fn_table[1,i] = np.std(fn_aux)
    

print "Statistics: (mean and std)"
print ""
print "True positive"
print vp_table
print "True Negative"
print vn_table
print "False Positive"
print fp_table
print "False Negative"
print fn_table
